<a href="https://colab.research.google.com/github/baichen-du/Kaggle/blob/main/test2/Copy_of_Creating%2C_Reading_and_Writing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240821%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240821T061419Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4abc822d08b470169172e59abe3f50e809b23c61f9c927d945cd822fe277b896affc4368783580f603941a296a621ce3a7671edb3450f05efc97ed75807fdab908fc2f3a450722cdbc0bbc49b9d092dc4567f96992bf247a973ab7a748becce72794ce95533726fcab58a55e4978e1354a0df24d2589eb925f98cae9e2ed2c2736151712b212e7481ae84a3d677b88981528ad11a3e241997e42ced3e440732b5618a42b3f2022e7791d6b686ca42558885283b739ababdcf584f984402b47a8369ed30fe63532613b869a082ce46ab4ec0f70bb2c2d5426f4e6c31770fb1394a2c04f5d787685953fcf72359a50a2e4bab133beee7075d0c969f14158274a8c,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240821%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240821T061419Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5b0a7048f4eed5356e70a46ebeb5297715ea7dddece2c7d8dd1bcd7eaed61e79465fa9ea00537c24d16f1a9bf01676635698532a7a94fe5b4f50875af6a90b4ebbed2c6add8e0eac4f25b31a6d475cfab8b22b2320241a1b47c14b1406142bb5211d41e04b060958cab2e343ec954ec0ca37b16129795227c470605b835717fe4a3882caa697cf1d52d3c6944f7cd2add304dec47f854f655a2feb7f5ee8cbb17ddcb2414c7f38bcf8efebeae3186d9c96e4b2b00ce0233a6e2a08158fa18b69c163c11a9ce7fb4d1b34ba05b6ad5b78c8cd95cb661d4a1256904ea5c312812e3b4ab021730d66796121afdfe9a869f3cc65891fe5235b43dd69520b8d7ac084,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240821%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240821T061419Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4ae823c8b851db469c0eb3af34b108c6d4d140e5057e5faba7dd8df6877c55268051d6efea8990f3becb89f735b12107d6e6d9780dfe298c12a157dc050cbc0771aa31cdc4c868a207505a8dfb6d4c70dfbe2f09f6ac35145d0ce78bd8a3a368d4f2fa7b1e15023cec7132649e9ae4f7a92e1e9ea33779d5db88f7da21df642e2757a2cff08b8478b877156d87eca0ae7de7ed2c4f221407deebcca59df3cc4a8d84f3f51ce1a86c1d5e6d4ddfb40a042f635bef3a22bce69a2cc1516650ac28cea35904db08ba41d0c7154955c0803523186ee5e0971e370c2d7ae81970863c134a157b821504b186c166aa9d3673bb1119324308689de9a5d1a6504d77562b,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240821%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240821T061419Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1baaa8ccb3f39df232fb087f58c92ab806ad5074f964e19d338d176823ba8477b0bd3dd9066226ff07b3e62a32e7284518ae5d3059b3bf663b3b6e91f8b2aed10f610918e7ff8df6e3705fcd4b69f6ed840ea73730fbc9cba22ec74c56bff874c4ade6e00194b7dd67d823c93efa2b39ac64c8b60db3146b60fab02a590d16dddf951a1c6500d270b16372a7acfbd26668dd1d554181d1bfd60303579edb515f6888d9e0d9316a26500db7f674fd09a2a9bbfc5a664d7d59c49a136913b94df92b184ba95242c0a8e76c2d4b9113f04b6fe796d171654b3c8744e51ed49adcd4d0c8318cd8ab6087ba0f1c356b8f22724be8588fe9331e68b3fd2a008665c158,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240821%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240821T061420Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D85f41e2b024c0ecd439684d9f83937c15caa08ac02d601076d4183c76f4f29841b56213896d48327e5dbc71eb3e0b93273b0b9cf598d5577d2e218101b17e913d7d653d719173ce76138d259b9fc575d090b83109d498d65f90efe8f9d13240f44a00249accf3d02eed041ac3c5006c1ed1e6c261907c4e61a3ab69c80f51ef6ecd9dca8020c2524ad00d54fa73776cfd6bad881fc3ae808ed047d739e66f02aa00b9b8fcc1f05786e580e390b722c08df418836c96f32d800c9400ac2562cd3813874f6d34e0de9a027b08bc931ebff3da4bd42a6cc77ab8a0b990c7728d092b4c164bd8ca6c3b0e4b54f00d0ab3448ac69b3272fdb988e274f09486182e6ac,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240821%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240821T061420Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D59e7d20ae6cbc8b72b33656976d59cceabdb31f3b0034059f31e1dbcdc7d27ddb11b9dba7ddb9efb888dec62d7077a2079ea489c128c6c49ab1ab42e06540c9e114373541b641e96dc1addf5bcce009adca8e92f5ef5307fce23287bb182466baaa4c70b4ca84431dbd04a4b9c7c8e3ae673d4eb228f2f18fc2ec00689658e4aa61fe8d223d7d2e19ed78baf2c0682e406b0e46cf9f21577e754777c8ddf9a5834acb2b5a182de9adfa093c6d36a664e8f5092161e084508569d7112e8ab870411a815d38a0766a70406ccfb4fe6788dc56133e1b7521be5c7c3ffd24c8b1779733a64ac86309a3878f38c765643184ac3679f1fe89a9a20ff9afee301400007,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240821%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240821T061420Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da04ee9f92804b91f357eda1a91f373fab51c25861a7caf5c88bf290ae6a3cc3a9967a69dc9f910f2dab2a09696cbf8074ffca517382e376d6e80a753bf3f0ef94202e42c797e0b6d75a181176633f8c4681e68bb2f99bb039a4965bc03895dc66093db00b2ebd47b995eed563304e5a680d88cef7da564ef0e22abb7cc765cfc9009b7d7d1d987262060f82c37b8d6d5dc62229494de55cd6276c2a11bf37af48dae69c0dc4760ef4bb2a3725042ec2ba855fada83822aadbcb725faafdd85d29917b204810db927460c5169cdc727f228c2a7ef468784c3b9a74fde828aad81f3f304be1a8cbc89fafb834b078af1f95a374e5639bcfc388a2f473dbaf63a70,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240821%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240821T061420Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D169584c212c95321d8f7223515521e698010b0997a25f03d59efc78e5577b8823be11ea4e38a6b5be56a88628cfd1d4dbe3e56f608306dd26dcd7bd55eafc5243ae64b3db2fd2e10017dafadd6113584293182f672fabec141ba2d4a6aa353be72b7a6a45ac56ec3b978bdba956ddb75c5f26a055da6c2542cc459a57d8f97d778b3f32f2dfd7492125c542c85ff95985857c4b10bb8a1a5f77338dc90f520ff80178a620507a95d0063973bf6abcfa07f9d4434ef4775443ccb467ab290131cdd32d8b0b962a86838fcab9e0d90e3139f5879fee84cbac04053730395c5c7728a6285674b4e23d250c348c085394dff0d43a3df5343eabe8d14c946778fd048,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240821%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240821T061420Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D496a95679f021c6f8d1417ddd66a16277a7c2cb358fbc861c18702b711231d2a21676b82aa9ed144b1ea0726156a46247b8c8360d85a943a9e4db5f5c78e4a897c8121994cf473880f8bbd240c4972409e55d622878c904b7703ce6098ee831f86a6d32c4e52ad274419f44e4ace47a8f7e937746ffc2a81b5c16620722fb5875346e34f1b8b4b1bfb78198c03d01e2879da69311feeb9f831de483ee3c60684a7f84fab44a222afb02006884426a151e55b8136969d421a3e43d71415f40f436a71e816df4db69ffd3a6253c3905fc58fb6e0462b9bd56fb69a18a1fd857d2bf8df23d468f71bceab47af6a84455b91cedd6672142e9a2664484ae5162f81a3,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240821%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240821T061420Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D510a4877cfddb96983ee6c89436270fc34fd03e2f11574a9ae6591c87fca4358f6962c76623ac8f3d1cf7319dd2a7e386e523eb094d4cd5bdb0812054e835dacfba92d76feb4ee99edbe949def619f37f7e6d28fabe98661ea6d91eb285f2f0bf4e41d7b9eb702056696ccabb9d7deaf8e0910b985f18e575812b4bd864098f8b7950e49636ba23f26061c351d0331a55db91f591c31f9c7ea4233788fee11e33aa12302d3ea6c88c98ade726c89d5e8aaddc517c2d28cd7d872ba1bb693cdf4aa552127015b05ec2f04262ed3c9d03af8eef1758669e65d086f60c6032146b9b56c968097bda752f36984c15ba0d4ba530b99d181d504550b190aba14593281'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Introduction

In this micro-course, you'll learn all about **[pandas](https://pandas.pydata.org)**, the most popular Python library for data analysis.

Along the way, you'll complete several hands-on exercises with real-world data.  We recommend that you work on the exercises while reading the corresponding tutorials.  

**To start the first exercise, please click [here](https://www.kaggle.com/kernels/fork/587970).**

In this tutorial, you will learn how to create your own data, along with how to work with data that already exists.  

# Getting started

To use pandas, you'll typically start with the following line of code.

In [ ]:
import pandas as pd

# Creating data

There are two core objects in pandas: the **DataFrame** and the **Series**.

### DataFrame

A DataFrame is a table. It contains an array of individual *entries*, each of which has a certain *value*. Each entry corresponds to a row (or *record*) and a *column*.

For example, consider the following simple DataFrame:

In [ ]:
pd.DataFrame({'Yes': [50, 21], 'No': [131, 2]})

In this example, the "0, No" entry has the value of 131. The "0, Yes" entry has a value of 50, and so on.

DataFrame entries are not limited to integers. For instance, here's a DataFrame whose values are strings:

In [ ]:
pd.DataFrame({'Bob': ['I liked it.', 'It was awful.'], 'Sue': ['Pretty good.', 'Bland.']})

We are using the `pd.DataFrame()` constructor to generate these DataFrame objects. The syntax for declaring a new one is a dictionary whose keys are the column names (`Bob` and `Sue` in this example), and whose values are a list of entries. This is the standard way of constructing a new DataFrame, and the one you are most likely to encounter.

The dictionary-list constructor assigns values to the *column labels*, but just uses an ascending count from 0 (0, 1, 2, 3, ...) for the *row labels*. Sometimes this is OK, but oftentimes we will want to assign these labels ourselves.

The list of row labels used in a DataFrame is known as an **Index**. We can assign values to it by using an `index` parameter in our constructor:

In [ ]:
pd.DataFrame({'Bob': ['I liked it.', 'It was awful.'],
              'Sue': ['Pretty good.', 'Bland.']},
             index=['Product A', 'Product B'])

### Series

A Series, by contrast, is a sequence of data values. If a DataFrame is a table, a Series is a list. And in fact you can create one with nothing more than a list:

In [ ]:
pd.Series([1, 2, 3, 4, 5])

A Series is, in essence, a single column of a DataFrame. So you can assign row labels to the Series the same way as before, using an `index` parameter. However, a Series does not have a column name, it only has one overall `name`:

In [ ]:
pd.Series([30, 35, 40], index=['2015 Sales', '2016 Sales', '2017 Sales'], name='Product A')

The Series and the DataFrame are intimately related. It's helpful to think of a DataFrame as actually being just a bunch of Series "glued together". We'll see more of this in the next section of this tutorial.

# Reading data files

Being able to create a DataFrame or Series by hand is handy. But, most of the time, we won't actually be creating our own data by hand. Instead, we'll be working with data that already exists.

Data can be stored in any of a number of different forms and formats. By far the most basic of these is the humble CSV file. When you open a CSV file you get something that looks like this:

```
Product A,Product B,Product C,
30,21,9,
35,34,1,
41,11,11
```

So a CSV file is a table of values separated by commas. Hence the name: "Comma-Separated Values", or CSV.

Let's now set aside our toy datasets and see what a real dataset looks like when we read it into a DataFrame. We'll use the `pd.read_csv()` function to read the data into a DataFrame. This goes thusly:

In [ ]:
wine_reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv")

We can use the `shape` attribute to check how large the resulting DataFrame is:

In [ ]:
wine_reviews.shape

So our new DataFrame has 130,000 records split across 14 different columns. That's almost 2 million entries!

We can examine the contents of the resultant DataFrame using the `head()` command, which grabs the first five rows:

In [ ]:
wine_reviews.head()

The `pd.read_csv()` function is well-endowed, with over 30 optional parameters you can specify. For example, you can see in this dataset that the CSV file has a built-in index, which pandas did not pick up on automatically. To make pandas use that column for the index (instead of creating a new one from scratch), we can specify an `index_col`.

In [ ]:
wine_reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)
wine_reviews.head()

# Your turn

If you haven't started the exercise, you can **[get started here](https://www.kaggle.com/kernels/fork/587970)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*